# Big Data Mining, 1st Assignment solution
## Vangelis Christou, p2821805



In [ ]:
#read
import pandas as pd 
df = pd.read_csv("groceries.csv",sep=";") 

#preview data
df.head(20)


In [80]:
#view default  data types
df.dtypes

Customer_ID                int64
Age                        int32
Sex                     category
Marital_Status          category
Education               category
Income                     int32
Customer_Rating         category
Persons_in_Household       int64
Occupation              category
Groceries                 object
Customer_Rating_enum       int32
Education_enum             int32
dtype: object

In [ ]:
import numpy as np
#replace ' ' values with NaN
df = df.replace(r'^\s*$', np.nan, regex=True)

#set proper data types
#numeric
df["Age"] = pd.to_numeric(df["Age"],downcast='integer')
df["Income"] = pd.to_numeric(df["Income"],downcast='integer')
df["Persons_in_Household"] = pd.to_numeric(df["Persons_in_Household"])

#nominal
df["Sex"] = df["Sex"].astype('category')
df["Marital_Status"] = df["Marital_Status"].astype('category')
df["Occupation"] = df["Occupation"].astype('category')

#ordinal
#for the ordinal columns we will replace with an enumeration to calculate similarity as numeric

cRatings = df["Customer_Rating"] = pd.Categorical(df["Customer_Rating"], categories=["poor", "fair", "good", "very_good", "excellent"], ordered=True)

cRatings = cRatings.replace("poor",1).replace("fair",2).replace("very_good",4).replace("good",3).replace("excellent",5)
df["Customer_Rating_enum"] = cRatings.astype(np.int)


cEducation = df["Education"] = pd.Categorical(df["Education"], categories=["primary", "secondary", "tertiary"], ordered=True)
cEducation = cEducation.replace("primary",0).replace("secondary",1).replace("tertiary",2)
df["Education_enum"] = cEducation.astype(np.int)



#fill Nan values with mean
df.fillna(df.mean().astype(int), inplace=True)
df["Age"] = df["Age"].astype(np.int) 
df["Income"] = df["Income"].astype(np.int) 

# convert Groceries to set of values
df.Groceries= df.Groceries.str.split(",") 

df.dtypes





In [ ]:

df.head(15)

In [ ]:
import itertools
import time

def jaccard_sim(list1, list2):
    intersection = len(list(set(list1).intersection(list2)))
    union = (len(list1) + len(list2)) - intersection
    return round(float(intersection) / union, 2)

def numeric_sim(num1, num2, maxvalue,minvalue):
    return 1- abs(num1-num2) / (maxvalue - minvalue)

def nominal_sim(nom1,nom2):
    return int(nom1 == nom2)


   

def pairwiseJaccard(cust):
    sim={}
    print("Computing Jaccard similarities")
    start_time = time.time()
    for pair in itertools.combinations(cust, r=2):
        s=jaccard_sim(cust[pair[0]],cust[pair[1]])
        sim[tuple([pair[0],pair[1]])]=s
    
    end_time = time.time()
    print("Jaccard Success after: " ,end_time-start_time)
    return sim

def pairwiseNominal(cust):
    sim={}
    print("Computing Nominal similarities")
    start_time = time.time()
    for pair in itertools.combinations(cust, r=2):
        s=nominal_sim(cust[pair[0]],cust[pair[1]])
        sim[tuple([pair[0],pair[1]])]=s
    
    end_time = time.time()
    print("Nominal Success after: " ,end_time-start_time)
    return sim

def pairwiseNumeric(cust, maxvalue,minvalue):
    sim={}
    print("Computing numeric similarities")
    start_time = time.time()
    for pair in itertools.combinations(cust, r=2):
        s=numeric_sim(cust[pair[0]],cust[pair[1]],maxvalue,minvalue)
        #print(cust[pair[0]],cust[pair[1]],maxvalue,minvalue,"=", s,"\n")
        sim[tuple([pair[0],pair[1]])]=s
    
    end_time = time.time()
    print("Numeric Success after: " ,end_time-start_time)
    return sim


In [ ]:
#numeric
age_d = dict(zip(df.Customer_ID, df.Age))
persons_d = dict(zip(df.Customer_ID, df.Persons_in_Household))
income_d = dict(zip(df.Customer_ID, df.Income))

#nominal
sex_d = dict(zip(df.Customer_ID, df.Sex))
occupation_d = dict(zip(df.Customer_ID, df.Occupation))
marital_d = dict(zip(df.Customer_ID, df.Marital_Status))

#ordinal
customer_rating_d = dict(zip(df.Customer_ID, df.Customer_Rating_enum))
education_enum_d = dict(zip(df.Customer_ID, df.Education_enum))

#set
groceries_d = dict(zip(df.Customer_ID, df.Groceries))


#helper test dictionaries
#short_groceries_d = dict(list(groceries_d.items())[0:100])
#short_sex_d = dict(list(sex_d.items())[0:100])
#short_cust_rating = dict(list(customer_rating_d.items())[0:100])




In [ ]:
###Calculate Similarities

#numeric
simAge = pairwiseNumeric(age_d)
simHouseholdPersons = pairwiseNumeric(persons_d)
simIncome = pairwiseNumeric(income_d)

#nominal
simSex = pairwiseNominal(sex_d)
simOccupation = pairwiseNominal(occupation_d)
simMaritalStatus = pairwiseNominal(marital_d)


#ordinal
simCustomerRating = pairwiseNumeric(customer_rating_d,df.Customer_Rating_enum.max(), df.Customer_Rating_enum.min())
simEducation = pairwiseNumeric(education_enum_d,df.Education_enum.max(), df.Education_enum.min())

#set
simGroceries = pairwiseJaccard(groceries_d)





In [ ]:
import operator

def GetSimilarCustomers(customerID,head,sim):
    filtered_dict = {k:v for (k,v) in sim.items() if customerID in k}
    sorted_d = dict(sorted(filtered_dict.items(), key=operator.itemgetter(1),reverse=True))
    return dict(list(sorted_d.items())[0:head])


def GetDictionaryWithAvgSimilarity(listSim):
    d0=listSim[0]
    i=1;
    while i < len(listSim) :
        for k, value in d0.items():
            d0[k] = float((d0[k] + listSim[i][k]) / 2)

        i+=1
    return d0



In [ ]:


#SimilarityList = [simGroceries, simSex]

SimilarityList = [simAge,simHouseholdPersons,simIncome, simMaritalStatus, simGroceries, simSex,simOccupation, simCustomerRating, simEducation]



avgSim =  GetDictionaryWithAvgSimilarity(SimilarityList)

customerids = [73]
for i in customerids: 
    print(GetSimilarCustomers(i,10,avgSim))
    print("----\n")


#73, 563, 1603, 2200, 3703, 4263, 5300, 6129, 7800, 8555

  
# printing final result 



